In [ ]:
%pip install llama-index llama-index-vector-stores-upstash upstash-vector

In [ ]:
import setup
setup.init_django()

In [ ]:
django_pdf = "data/django.pdf"

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore #changed from the llamaINdex DOcs

In [ ]:
reader = SimpleDirectoryReader(input_files=[django_pdf])

In [ ]:
import helpers
import openai
openai.api_key = helpers.config("OPENAI_API_KEY", default=None)

In [ ]:
url = helpers.config("UPSTASH_VECTOR_URL", default = None)
token = helpers.config("UPSTASH_VECTOR_TOKEN", default = None)
url

In [ ]:
data = reader.load_data()


In [ ]:
import httpx

timeout = httpx.Timeout(600.0, connect = 600.0)
client = httpx.Client(timeout=timeout)

In [ ]:
from upstash_vector import Index

upstash_index = Index(
            url=url,
            token=token,
            retries =5,
            retry_interval =0.2
             )
upstash_index._client = client

In [ ]:
vector_store = UpstashVectorStore(
    url = url,
    token=token,
)

vector_store._index = upstash_index

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(data, storage_context, insert_batch_size= 500)

In [ ]:
index_persist = VectorStoreIndex.from_vector(vector_store= vector_store)

In [ ]:
query_engine = index_persist.as_query_engine()

In [ ]:
r = query_engine.query("What is prefetch_related? Show an example usage with a QuerySet")

In [ ]:
print(r.response)